In [ ]:
#For F1 

#THE DRIVERS (AND THEIR TOTAL TIME) THAT WOULD HAVE BEEN PASSED BY DRIVER WITH id=20(CAN BE ANY DRIVER) IN RACE WITH ID=841(CAN BE ANY RACE), IF HE HADN'T STOPPED AT ANY PITSTOP

query = """
prefix lp: <http://example.org/motor-sports/formula-one/lap_times/> 
prefix ps: <http://example.org/motor-sports/formula-one/pit_stops/> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT  (STR(REPLACE(STR(?l), ".*_", "")) AS ?RiderId)
       (COALESCE(SUM(IF(isLiteral(?otherLapTime) && xsd:float(?otherLapTime), xsd:float(?otherLapTime), 0)), 0) AS ?totalRaceTime)
      
		
WHERE {
  
 
  {
    ?l <http://example.org/motor-sports/formula-one/lap_times/msec> ?laptimeBnode.
    ?laptimeBnode ?otherLapTimeProperty ?otherLapTime.
    FILTER (CONTAINS(STR(?otherLapTimeProperty), "RDF.")&& ?l != lp:841_17  && CONTAINS(STR(?l), "http://example.org/motor-sports/formula-one/lap_times/841"))
  }
    {
    SELECT (COALESCE(SUM(IF(isLiteral(?a) && xsd:float(?a), xsd:float(?a), 0)), 0) AS ?OurTargetRacerTotalTimeInThatCircuit)
    WHERE {
      VALUES ?l {lp:841_17}
      ?l <http://example.org/motor-sports/formula-one/lap_times/msec> ?laptimeBnode.
      ?laptimeBnode ?p ?a .
      BIND(REPLACE(?a, ",", ".") AS ?a)
      FILTER (CONTAINS(STR(?p), "RDF."))
    }
  }
 {
    SELECT (COALESCE(SUM(IF(isLiteral(?k) && xsd:float(?k), xsd:float(?k)*1000, 0)), 0) AS ?OurTargetRacerTotalPitStopDurationInThatCircuit)
    WHERE {
      VALUES ?l {ps:841_17}
      ?l ?pslapstop ?k_raw.
  BIND(REPLACE(?k_raw, ",", ".") AS ?k)
  FILTER (CONTAINS(STR(?pslapstop), "duration"))
    }
  }
}
GROUP BY ?l ?OurTargetRacerTotalTimeInThatCircuit ?OurTargetRacerTotalPitStopDurationInThatCircuit
HAVING ((?totalRaceTime < ?OurTargetRacerTotalTimeInThatCircuit) && (?totalRaceTime > ?OurTargetRacerTotalTimeInThatCircuit - ?OurTargetRacerTotalPitStopDurationInThatCircuit ))
"""

#QUALIFYING WINNERS, NUMBER OF TIMES PER YEAR

query = """    PREFIX q: <http://example.org/motor-sports/formula-one/qualifying/>
    PREFIX d: <http://example.org/motor-sports/formula-one/driver/>
    PREFIX rc: <http://example.org/motor-sports/formula-one/races-circuits/>
    PREFIX FOAF: <http://xmlns.com/foaf/0.1/>
    prefix f1: <http://example.org/motor-sports/formula-one/>

    SELECT (CONCAT(?firstname, "_", ?surname) AS ?full_name) ?inYear (GROUP_CONCAT(?location; SEPARATOR=", ") AS ?locations) (COUNT(?qualifyingResult) AS ?qualifyingWins)
    WHERE {
    ?qualifyingResult a f1:Qualifying;
        q:hasConstructorID ?co_id;
        q:hasDriverID ?d_id;
        q:hasRaceID ?rc_id;
        q:position "1".

    ?rc_id a f1:F1RaceEvents;
        rc:inYear ?inYear;
        rc:location ?location.

    ?d_id a f1:Driver;
        FOAF:firstName ?firstname;
        FOAF:surname ?surname.
}
GROUP BY ?inYear ?firstname ?surname ?locations ?qualifyingWins
ORDER BY DESC(?inYear)
LIMIT 50
"""

#SPRINT WINNERS IN ALL SEASONS

query = """
  PREFIX s_res: <http://example.org/motor-sports/formula-one/sprint_results/>
  PREFIX rc: <http://example.org/motor-sports/formula-one/races-circuits/>
  PREFIX d: <http://example.org/motor-sports/formula-one/driver/> 
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX FOAF: <http://xmlns.com/foaf/0.1/> 
PREFIX f1: <http://example.org/motor-sports/formula-one/>

  SELECT ?location (CONCAT(?firstname,"_",?surname) AS ?full_name) ?inYear ?time
  WHERE{
      _:x a f1:SprintResult;
      s_res:hasConstructorID ?co_id;
      s_res:hasDriverID ?d_id;
      s_res:hasRaceID ?rc_id;
      s_res:position "1";
      s_res:time ?time.

      ?rc_id a f1:F1RaceEvents;
      rc:location ?location;
      rc:inYear ?inYear.

      ?d_id a f1:Driver;
      FOAF:firstName ?firstname;
      FOAF:surname ?surname.
  }
GROUP BY ?location ?firstname ?surname ?inYear ?time
ORDER BY DESC(?inYear)
LIMIT 10
"""
 
#WHICH DRIVERS DROVE FOR A CONSTRUCTOR FROM THEIR COUNTRY AND WHAT YEARS

query = """
   PREFIX co: <http://example.org/motor-sports/formula-one/constructors/>
  PREFIX rc: <http://example.org/motor-sports/formula-one/races-circuits/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX res: <http://example.org/motor-sports/formula-one/results/>
  PREFIX FOAF: <http://xmlns.com/foaf/0.1/>
  PREFIX d: <http://example.org/motor-sports/formula-one/driver/>
prefix f1: <http://example.org/motor-sports/formula-one/>


 SELECT (CONCAT(?firstname, "_", ?surname) AS ?full_name) ?co_name ?co_country (GROUP_CONCAT(DISTINCT ?year; SEPARATOR=", ") AS ?list_of_years)
WHERE {
  _:bo a f1:Result;
       res:hasConstructorID ?co_id;
       res:hasDriverID ?d_id;
       res:hasRaceID ?rc_id.

  ?co_id a f1:F1Constructor;
         co:originalFrom ?co_country;
         co:name ?co_name.

  ?d_id a f1:Driver;
        d:nationality ?d_country;
        FOAF:firstName ?firstname;
        FOAF:surname ?surname.

  ?rc_id a f1:F1RaceEvents;
         rc:inYear ?year.

  FILTER(STR(?co_country) = STR(?d_country))
}
GROUP BY ?firstname ?surname ?co_name ?co_country 
"""

#NUMBER OF DISQUALIFICATIONS FOR A DRIVER PER SEASON

query = """
PREFIX d: <http://example.org/motor-sports/formula-one/driver/> 
PREFIX rc: <http://example.org/motor-sports/formula-one/races-circuits/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX res: <http://example.org/motor-sports/formula-one/results/> 
prefix f1: <http://example.org/motor-sports/formula-one/>


SELECT ?dRef ?year (COUNT(?rID) as ?NumOfDisqualification)
WHERE {
_:b0 a f1:Result ;
        res:hasDriverID ?dID ;
        res:hasRaceID ?rID ;
        res:status "Disqualified" .
?rID a f1:F1RaceEvents ;
       rc:inYear ?year .
?dID a f1:Driver ;
       d:driverRef ?dRef .
}
GROUP BY ?dRef ?dID ?year 
ORDER BY DESC (?year)

limit 5
"""

#CONSTRUCTOR WINNERS PER SEASON

query = """
PREFIX d: <http://example.org/motor-sports/formula-one/driver/> 
PREFIX rc: <http://example.org/motor-sports/formula-one/races-circuits/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX res: <http://example.org/motor-sports/formula-one/results/> 
PREFIX co: <http://example.org/motor-sports/formula-one/constructors/>
prefix f1: <http://example.org/motor-sports/formula-one/>

SELECT ?cRef ?maxG ?year
WHERE{
{
SELECT ?cRef (MAX(?g) as ?maxG) ?year
WHERE{
SELECT ?cRef (SUM(xsd:float(?p)) as ?g) ?year #Getting all the constructors with their total points per season
WHERE {
_:b0 a f1:Result ;
        res:hasRaceID ?rID ;
        res:hasConstructorID ?cID ;
        res:points ?p .

?rID a f1:F1RaceEvents ;
    rc:inYear ?year .
?cID a f1:F1Constructor ;
        co:constructorRef ?cRef .
    }GROUP BY ?year ?cRef
}GROUP BY ?year ?cRef
ORDER BY DESC(?year) DESC(?maxG)
}


FILTER NOT EXISTS {

{
SELECT ?cRef2 (MAX(?g2) as ?maxG2) ?year2
WHERE{
SELECT ?cRef2 (SUM(xsd:float(?p2)) as ?g2) ?year2 #Getting all the constructors with their total points per season
WHERE {
_:b1 a f1:Result ;
        res:hasRaceID ?rID2 ;
        res:hasConstructorID ?cID2 ;
        res:points ?p2 .

?rID2 a f1:F1RaceEvents ;
    rc:inYear ?year2 .
?cID2 a f1:F1Constructor ;
        co:constructorRef ?cRef2 .
    }GROUP BY ?year2 ?cRef2
}GROUP BY ?year2 ?cRef2
}
FILTER (?cRef != ?cRef2 && ?maxG2 > ?maxG && ?year = ?year2)
}
}
GROUP BY ?cRef ?maxG ?year
"""

In [ ]:
#FOR MOTOGP

#TOP SIX RIDERS WITH MOST WINS IN THEIR COUNTRY

query = """ PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX gp: <http://example.org/motor-sports/motoGP/> 
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT ?rider (COUNT(?rider) AS ?winCount) (GROUP_CONCAT(DISTINCT ?info; separator=", ") AS ?MoreInfoAboutTheDriver)
WHERE {
    ?MotoRaceEvents gp:Has ?b .
    ?b gp:TeamRider ?rider ;
       gp:WinnerRiderNationality ?Ridernat ;
       gp:Held_On ?track .
    ?track gp:country ?country .
    ?b gp:TeamRider/rdfs:seeAlso ?info .
  
    FILTER(?Ridernat=?country)      
} 

GROUP BY ?rider
HAVING (COUNT(?rider) >= 0)
ORDER BY DESC(?winCount)
LIMIT 6
"""

#TOP 5 CONSTRUCTORS WITH THE MOST WINS BEFORE 2005 IN 250CC CLASS WITH DRIVERS FROM ITALY

query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX gp: <http://example.org/motor-sports/motoGP/> 
prefix foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT  ?constructor (COUNT(?constructor) AS ?conswinCount) 
WHERE {
  
                
  

    ?MotoRaceEvents250cc gp:Has ?b .
  
    ?b gp:InCategory "250cc" ;
       gp:WinnerRiderNationality ?Ridernat ;
       gp:HasWinnerTeam ?constructor ;
       gp:season ?season .
  
  
 FILTER(xsd:integer(?season) < 2005 && ?Ridernat = "IT")

} 


GROUP BY ?constructor 
HAVING (COUNT(?constructor) >= 0)
ORDER BY DESC(?conswinCount)
LIMIT 5 
"""

#WHICH COUNTRY HAS THE RIDERS WITH THE MOST WINS IN TOTAL, HOW MANY ARE THE WINS AND THE RIDERS

query = """
 PREFIX ns1: <http://example.org/motor-sports/motoGP/>

  SELECT ?nationality (SUM(?num) As ?numofwins) (COUNT(?rider) As ?numofriders)
  WHERE{
  ?rider a ns1:Rider;
        ns1:RiderNationality ?nationality;
        ns1:HadFirstPlaces ?num.

  }
  GROUP BY ?nationality
  ORDER BY DESC (?numofwins)
  LIMIT 1
  """

In [ ]:
#MOTOGP AND F1 CONSTRUCTORS

query = """  PREFIX co: <http://example.org/motor-sports/formula-one/constructors/>
  PREFIX ns1: <http://example.org/motor-sports/motoGP/>
  prefix f1: <http://example.org/motor-sports/formula-one/>

SELECT ?name1
WHERE {
  ?f1Constructor a f1:F1Constructor;
                co:name ?name1.

  ?motoGPConstructor a ns1:MotoGPConstructor;
                ns1:HasName ?name2.
  
  FILTER(STR(?name1)=STR(?name2))
}
LIMIT 10
"""